In [4]:
import tensorflow as tf
import pathlib
import pandas as pd
from PIL import Image
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras import layers, optimizers, regularizers

# Directories for training, validation, and test data
train_dir = 'train'
val_dir = 'validation'
test_dir = 'test'

# Image size and batch size
img_size = (224, 224)  # Changed to 224x224 for DenseNet compatibility
batch_size = 16  # Reduced batch size to fit in memory

# Load datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

# Data normalization function
def norm_data(ds):
    prepr_layer = tf.keras.layers.Rescaling(1./255)
    norm_ds = ds.map(lambda x, y: (prepr_layer(x), y))
    return norm_ds

# Data augmentation function
def augment_data1(ds, crop_size):
    layer1 = tf.keras.layers.CenterCrop(crop_size, crop_size)
    aug_ds = ds.map(lambda x, y: (layer1(x), y))

    layer2 = tf.keras.layers.RandomRotation(factor=(-0.05, 0.05))
    aug_ds = aug_ds.map(lambda x, y: (layer2(x), y))

    layer3 = tf.keras.layers.RandomZoom(height_factor=(-0.1, -0.01), width_factor=(-0.1, -0.01))
    aug_ds = aug_ds.map(lambda x, y: (layer3(x), y))

    return aug_ds

# Image height for augmentation
img_height = 244

# Augment and normalize datasets
train_ds_aug = augment_data1(train_ds, img_height - 20)
train_ds_norm = norm_data(train_ds_aug)

val_ds_aug = augment_data1(val_ds, img_height - 20)
val_ds_norm = norm_data(val_ds_aug)

test_ds_aug = augment_data1(test_ds, img_height - 20)
test_ds_norm = norm_data(test_ds_aug)

# Autotune for performance optimization
AUTOTUNE = tf.data.AUTOTUNE
train_ds_norm = train_ds_norm.cache().prefetch(buffer_size=AUTOTUNE)
val_ds_norm = val_ds_norm.cache().prefetch(buffer_size=AUTOTUNE)
test_ds_norm = test_ds_norm.cache().prefetch(buffer_size=AUTOTUNE)

# Load DenseNet-201 model without classifier layers
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Set last 200 layers to be trainable
for layer in base_model.layers[-200:]:
    layer.trainable = True

# Add custom classification layers on top of the base model
flat = Flatten()(base_model.output)
drop1 = Dropout(0.4)(flat)
class1 = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(1e-8))(drop1)
drop2 = Dropout(0.4)(class1)
class2 = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-8))(drop2)
drop3 = Dropout(0.4)(class2)
class3 = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-8))(drop3)
drop4 = Dropout(0.4)(class3)
class4 = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(1e-8))(drop4)
drop5 = Dropout(0.4)(class4)
predictions = Dense(8, activation='softmax', kernel_regularizer=regularizers.l2(1e-8))(drop5)

# Define the complete model
model = Model(inputs=base_model.inputs, outputs=predictions)

# Compile the model
model.compile(optimizer=optimizers.Adam(learning_rate=2e-5),
              loss="categorical_crossentropy",
              metrics=['accuracy'])

# Train the model using the normalized training dataset and validate on validation dataset
history = model.fit(train_ds_norm, epochs=40, validation_data=val_ds_norm)

# Evaluate the model on the test set
model.evaluate(test_ds_norm)

# Make predictions on the test set
test_pred = model.predict(test_ds_norm)

# Save the trained model in h5 format
model.save("densenet201.h5", save_format='h5')

Found 5620 files belonging to 8 classes.
Found 1730 files belonging to 8 classes.
Found 680 files belonging to 8 classes.
Epoch 1/40
352/352 [==============================] - 136s 319ms/step - loss: 2.2755 - accuracy: 0.1835 - val_loss: 1.7253 - val_accuracy: 0.5127
Epoch 2/40
352/352 [==============================] - 82s 232ms/step - loss: 1.8017 - accuracy: 0.2952 - val_loss: 1.2885 - val_accuracy: 0.6451
Epoch 3/40
352/352 [==============================] - 82s 232ms/step - loss: 1.5581 - accuracy: 0.4075 - val_loss: 0.9146 - val_accuracy: 0.7185
Epoch 4/40
352/352 [==============================] - 82s 232ms/step - loss: 1.3419 - accuracy: 0.4762 - val_loss: 0.7245 - val_accuracy: 0.7613
Epoch 5/40
352/352 [==============================] - 81s 232ms/step - loss: 1.1390 - accuracy: 0.5617 - val_loss: 0.5123 - val_accuracy: 0.8046
Epoch 6/40
352/352 [==============================] - 81s 232ms/step - loss: 0.9420 - accuracy: 0.6436 - val_loss: 0.3852 - val_accuracy: 0.8948
Epoch 7